# Preámbulo

In [1]:
%pip install astropy==5.3.4
%pip install aplpy
%pip install matplotlib==3.9
%pip install ccdproc
%pip install photutils==1.13.0

In [2]:
import aplpy
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import wcs
from astropy.coordinates import SkyCoord
from scipy.optimize import curve_fit
from astropy.wcs import WCS
import re
import ccdproc as ccdp
from astropy.nddata import CCDData
from astropy import units as u
import photutils as pht
from photutils.profiles import RadialProfile
from scipy.ndimage import rotate

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def _to_identifier(label):
    """
    Convierte la etiqueta en un identificador Python válido.
    Reemplaza caracteres no alfanuméricos por guiones bajos.

    Parámetros:
      label: str, etiqueta a convertir.
    """
    # Reemplaza no alfanum por _.
    s = re.sub(r'\W', '_', str(label))
    if s and s[0].isdigit():
        s = '_' + s # La etiqueta no puede empezar con dígitos.
    return s or 'etiqueta'

def datos_fits(ruta_del_archivo, etiqueta_del_archivo, ext=1, register_globals=True, memmap=True):
    """
    Abre un FITS y crea variables globales nombradas con la etiqueta:
      hdu_{etiqueta}, header_{etiqueta}, data_{etiqueta}, w_{etiqueta}
    También retorna un dict con esas referencias.

    Parámetros:
      ruta_del_archivo: str, ruta al FITS.
      etiqueta_del_archivo: str, e.g.' _bias1'.
      ext: int, extensión a usar (por defecto 1).
      register_globals: bool, si: True, crea variables globales.
      memmap: bool, si: True, utiliza mapeo a memoria.

    Nota: cierra el HDUList con hdu_{etiqueta}.close() cuando termines.
    """
    etiqueta = _to_identifier(etiqueta_del_archivo)

    hdul = fits.open(ruta_del_archivo, memmap=memmap)
    try:
        hdu = hdul[ext]
    except IndexError:
        hdul.close()
        raise ValueError(f"El archivo no tiene la extensión {ext}.")

    header = hdu.header
    data = hdu.data
    w = WCS(header)

    if register_globals:
        g = globals()
        g[f"hdu_{etiqueta}"] = hdul
        g[f"header_{etiqueta}"] = header
        g[f"data_{etiqueta}"] = data
        g[f"w_{etiqueta}"] = w

    return

def imagen(data):
  '''
  Función que visualiza una imagen.
  '''
  f = aplpy.FITSFigure(data)
  f.show_grayscale()
  return

In [5]:
def primera_reduccion(hdu):
    '''
    Función que recorta el overscan de cada cuadrante, resta la media del overscan
    y escala por el gain de un archivo RAW y da como resultado la data completa del campo.

    Adicionalmente, muestra la imagen completa del campo.

    Parámetros:
        hdu: HDUList del archivo RAW cargado (e.g. hdu_raw_189)

    El resultado de esta función es un objeto CCDData.
    '''
    # ==================================================================================================
    # Se definen los límites de píxeles que deben tener las regiones de los cuadrantes según el archivo.
    # ==================================================================================================
    trimsec_data_r1 = hdu[1].header['DATASEC']
    trimsec_data_r2 = hdu[2].header['DATASEC']
    trimsec_data_r3 = hdu[3].header['DATASEC']
    trimsec_data_r4 = hdu[4].header['DATASEC']

    # =========================================
    # Se definen los extremos para cada región.
    # =========================================
    side1_r1 = (trimsec_data_r1.split(',')[0])[1::].split(':')
    side2_r1 = (trimsec_data_r1.split(',')[1])[:-1].split(':')

    side1_r2 = (trimsec_data_r2.split(',')[0])[1::].split(':')
    side2_r2 = (trimsec_data_r2.split(',')[1])[:-1].split(':')

    side1_r3 = (trimsec_data_r3.split(',')[0])[1::].split(':')
    side2_r3 = (trimsec_data_r3.split(',')[1])[:-1].split(':')

    side1_r4 = (trimsec_data_r4.split(',')[0])[1::].split(':')
    side2_r4 = (trimsec_data_r4.split(',')[1])[:-1].split(':')

    # =====================================================================================================
    # Se recortan los datos de cada región de tal forma que calcen con lo establecido en header['DATASEC'].
    # =====================================================================================================
    data_region1_recortada = hdu[1].data[int(side2_r1[0])-1:int(side2_r1[1]), int(side1_r1[0])-1:int(side1_r1[1])]
    data_region2_recortada = hdu[2].data[int(side2_r2[0])-1:int(side2_r2[1]), int(side1_r2[0])-1:int(side1_r2[1])]
    data_region3_recortada = hdu[3].data[int(side2_r3[0])-1:int(side2_r3[1]), int(side1_r3[0])-1:int(side1_r3[1])]
    data_region4_recortada = hdu[4].data[int(side2_r4[0])-1:int(side2_r4[1]), int(side1_r4[0])-1:int(side1_r4[1])]

    # ===============================================================================
    # Se definen las secciones que aportarán a la media de overscan para cada región.
    # ===============================================================================
    trimsec_overscan_r1 = hdu[1].header['BIASSEC']
    trimsec_overscan_r2 = hdu[2].header['BIASSEC']
    trimsec_overscan_r3 = hdu[3].header['BIASSEC']
    trimsec_overscan_r4 = hdu[4].header['BIASSEC']

    # =========================================
    # Se definen los límites de esas secciones.
    # =========================================
    s1_r1 = (trimsec_overscan_r1.split(',')[0])[1::].split(':')
    s2_r1 = (trimsec_overscan_r1.split(',')[1])[0:-1].split(':')

    s1_r2 = (trimsec_overscan_r2.split(',')[0])[1::].split(':')
    s2_r2 = (trimsec_overscan_r2.split(',')[1])[0:-1].split(':')

    s1_r3 = (trimsec_overscan_r3.split(',')[0])[1::].split(':')
    s2_r3 = (trimsec_overscan_r3.split(',')[1])[0:-1].split(':')

    s1_r4 = (trimsec_overscan_r4.split(',')[0])[1::].split(':')
    s2_r4 = (trimsec_overscan_r4.split(',')[1])[0:-1].split(':')

    # ================================
    # Se sustrae la media de OVERSCAN.
    # ================================
    data_region_1_sin_overscan = data_region1_recortada - hdu[1].data[int(s1_r1[0]):int(s1_r1[1]),int(s2_r1[0]):int(s2_r1[1])].mean()
    data_region_2_sin_overscan = data_region2_recortada - hdu[2].data[int(s1_r2[0]):int(s1_r2[1]),int(s2_r2[0]):int(s2_r2[1])].mean()
    data_region_3_sin_overscan = data_region3_recortada - hdu[3].data[int(s1_r3[0]):int(s1_r3[1]),int(s2_r3[0]):int(s2_r3[1])].mean()
    data_region_4_sin_overscan = data_region4_recortada - hdu[4].data[int(s1_r4[0]):int(s1_r4[1]),int(s2_r4[0]):int(s2_r4[1])].mean()

    # ======================================
    # Se define el GAIN para cada cuadrante.
    # ======================================
    gain_r1 = hdu[1].header['GAIN']
    gain_r2 = hdu[2].header['GAIN']
    gain_r3 = hdu[3].header['GAIN']
    gain_r4 = hdu[4].header['GAIN']

    # ===========================================================================
    # Se escala cada conjunto de datos en cada cuadrante por el gain de cada uno.
    # ===========================================================================
    data_region_1_con_gain = data_region_1_sin_overscan*gain_r1
    data_region_2_con_gain = data_region_2_sin_overscan*gain_r2
    data_region_3_con_gain = data_region_3_sin_overscan*gain_r3
    data_region_4_con_gain = data_region_4_sin_overscan*gain_r4

    # =====================
    # Se unen las regiones.
    # =====================
    f1 = np.hstack((data_region_2_con_gain, np.fliplr(data_region_3_con_gain)))
    f2 = np.hstack((np.flipud(data_region_1_con_gain), rotate(data_region_4_con_gain,180)))
    data_full = np.vstack((f1, f2))

    return data_full

In [6]:
def segunda_reduccion(data_full_raw,header_raw,ruta_bias,ruta_dark,ruta_skyflat):
    '''
    Función que realiza la segunda reducción de una imagen ya reducida en su
    primera reducción (recorte de overscan, resta de overscan y escala por gain).

    Esta función realiza:
        1. Resta el pedestal de BIAS.
        2. Resta el BIAS total.
        3. Resta el dark.
        4. Corrije por el flat.

    Parámetros:
        data_full: array 2D, data completa del campo en su primera reducción.
        ruta_bias: string, ruta al archivo FITS del BIAS.
        ruta_dark: string, ruta al archivo FITS del DARK.
        ruta_skyflat: string, ruta al archivo FITS del SKYFLAT.

    El resultado de esta función en un objeto CCDdata.
    '''
    # =================================
    # Sustracción del PEDESTAL DE BIAS.
    # =================================
    ccddata_bias  = CCDData.read(ruta_bias,hdu=1,unit='adu')
    biaslvl = ccddata_bias.header['BIASLVL']
    raw_sin_biaslvl = data_full_raw - biaslvl

    # =====================
    # Sustracción del BIAS.
    # =====================
    raw_sin_bias = raw_sin_biaslvl - ccddata_bias

    # =====================
    # Sustracción del DARK.
    # =====================
    exptime = header_raw['EXPTIME']
    ccddata_dark = CCDData.read(ruta_dark,hdu=1,unit='adu')
    raw_sin_dark = ccdp.subtract_dark(CCDData(raw_sin_bias, unit='adu'),
                                        ccddata_dark,dark_exposure=1*u.s,
                                        data_exposure=exptime*u.s,exposure_unit=u.s,
                                        scale=True)

    # =======================
    # Corrección por el FLAT.
    # =======================
    ccddata_flat = CCDData.read(ruta_skyflat,hdu=1,unit='adu')
    reducida = raw_sin_dark/ccddata_flat.data
    reducida = ccdp.flat_correct(raw_sin_dark,ccddata_flat,norm_value=1)

    imagen(reducida.data)

    return reducida

En este notebook se realizará la reducción de los datos del campo para cada archivo RAW.

# Archivos en distintos filtros

Hay 8 archivos RAW, de los cuales 4 están tomados en el filtro V y los otros 4 en el filtro B, esta naturaleza se determinará a partir de la descripción del header de cada archivo.

In [10]:
# Se definen las rutas de los archivos RAW y sus respectivas etiquetas.
ruta_raw_189 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0189-e00.fits.fz'
ruta_raw_190 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0190-e00.fits.fz'
ruta_raw_193 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0193-e00.fits.fz'
ruta_raw_194 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0194-e00.fits.fz'
ruta_raw_197 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0197-e00.fits.fz'
ruta_raw_198 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0198-e00.fits.fz'
ruta_raw_201 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0201-e00.fits.fz'
ruta_raw_202 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/RAWs/coj1m011-fa12-20241223-0202-e00.fits.fz'

rutas_raw = [ruta_raw_189,ruta_raw_190,ruta_raw_193,ruta_raw_194,ruta_raw_197,ruta_raw_198,ruta_raw_201,ruta_raw_202]
etiquetas_raw = ['raw_189','raw_190','raw_193','raw_194','raw_197','raw_198','raw_201','raw_202']

# Carga los datos de los archivos RAW.
for i in range(0,8):
  datos_fits(rutas_RAW[i],etiquetas_RAW[i],ext=0,memmap=False)

# Se definen listas que contienen los hdus, los headers y las datas de cada archivo RAW para poder acceder de manera simple.
hdus_raw = [hdu_raw_189,hdu_raw_190,hdu_raw_193,hdu_raw_194,
            hdu_raw_197,hdu_raw_198,hdu_raw_201,hdu_raw_202]
headers_raw = [header_raw_189,header_raw_190,header_raw_193,header_raw_194,
               header_raw_197,header_raw_198,header_raw_201,header_raw_202]
datas_raw = [data_raw_189,data_raw_190,data_raw_193,data_raw_194,
             data_raw_197,data_raw_198,data_raw_201,data_raw_202]

# Se muestra los filtros de cada archivo RAW.
for i in range(0,8):
  print(f'{etiquetas_raw[i]}: {headers_raw[i]['FILTER']}')

Set OBSGEO-B to   -31.272933 from OBSGEO-[XYZ].
Set OBSGEO-H to     1164.993 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -31.272933 from OBSGEO-[XYZ].
Set OBSGEO-H to     1164.993 from OBSGEO-[XYZ]'.


raw_189: B
raw_190: V
raw_193: B
raw_194: V
raw_197: B
raw_198: V
raw_201: B
raw_202: V


Notar que las numeraciones impares corresponden al filtro B, y las pares al filtro V.

# Reducción

Ahora, se hará la reducción de los archivos RAW del campo, el cual consistirá en dos secciones:
- Primera reducción:
    1. Se recortará cada cuadrante según lo indicado en el header sobre los pixeles,
    2. Se restará la media de overscan según el cuadrante de cada archivo.
    3. Se escalará cada cuadrante por el GAIN indicado por el header de cada uno.
    4. Se unirán los cuadrantes para formar la data del campo.
- Segunda reducción:
    1. Se restará el nivel de pedestal de BIAS según el archivo.
    2. Se restará el BIAS según el archivo.
    3. Se restará el DARK según el archivo.
    4. Se correjirá según el FLAT del archivo.

In [11]:
# Se define una lista que contiene la data de la primera reducción.

data_full_primera_reduccion = []

# Se aplica la primera reducción a cada archivo de datos.
for i in range (0,8):
  data_full_primera_reduccion.append(primera_reduccion(hdus_raw[i]))

In [12]:
# Se definen las rutas para los archivos que se usarán en la segunda reducción.
ruta_bias = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/Calibración/coj1m011-fa12-20241223-bias-bin1x1.fits.fz'
ruta_dark = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/Calibración/coj1m011-fa12-20241223-dark-bin1x1.fits.fz'
ruta_skyflat_V = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/Calibración/coj1m011-fa12-20241222-skyflat-bin1x1-V.fits.fz'
ruta_skyflat_B = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/Calibración/coj1m011-fa12-20241223-skyflat-bin1x1-B.fits.fz'

In [13]:
# Se definen las listas que contendrán los objetos CCDData que contienen la data de los campos reducidos.
campos_reducidos_filtro_B = []
campos_reducidos_filtro_V = []

# Se definen los iteradores con los índices pertenecientes a los archivos de cada filtro, donde, como se explicó
# anteriormente, los archivos con numeración impar pertenecen al filtro B, y los de numeración par, al filtro V.
iterador_filtro_B = [0,2,4,6]
iterador_filtro_V = [1,3,5,7]

# Campos reducidos en el filtro B.
for i in iterador_filtro_B:
  campos_reducidos_filtro_B.append(segunda_reduccion(data_full_primera_reduccion[i],headers_raw[i],ruta_bias,ruta_dark,ruta_skyflat_B))

# Campos reducidos en el filtro V.
for i in iterador_filtro_V:
  campos_reducidos_filtro_V.append(segunda_reduccion(data_full_primera_reduccion[i],headers_raw[i],ruta_bias,ruta_dark,ruta_skyflat_V))

Output hidden; open in https://colab.research.google.com to view.

# Comparación con los archivos BANZAI

Se realiza la comparación con los archivos BANZAI para cada reducción realizada.

In [14]:
# Se definen las rutas de los archivos BANZAI y sus respectivas etiquetas.
ruta_BANZAI_189 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0189-e91.fits.fz'
ruta_BANZAI_190 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0190-e91.fits.fz'
ruta_BANZAI_193 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0193-e91.fits.fz'
ruta_BANZAI_194 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0194-e91.fits.fz'
ruta_BANZAI_197 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0197-e91.fits.fz'
ruta_BANZAI_198 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0198-e91.fits.fz'
ruta_BANZAI_201 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0201-e91.fits.fz'
ruta_BANZAI_202 = '/content/drive/MyDrive/Colab Notebooks/Observacional VSCode/BANZAIs/coj1m011-fa12-20241223-0202-e91.fits.fz'

rutas_BANZAI = [ruta_BANZAI_189,ruta_BANZAI_190,ruta_BANZAI_193,ruta_BANZAI_194,
                ruta_BANZAI_197,ruta_BANZAI_198,ruta_BANZAI_201,ruta_BANZAI_202]

etiquetas_BANZAI = ['BANZAI_189','BANZAI_190','BANZAI_193','BANZAI_194',
                    'BANZAI_197','BANZAI_198','BANZAI_201','BANZAI_202']

# Carga de los datos de los archivos BANZAI.
for i in range(0,8):
  datos_fits(rutas_BANZAI[i],etiquetas_BANZAI[i])

# Se definen listas que contienen los hdus, los headers y las datas de cada archivo BANZAI
# en cada filtro para poder acceder de manera simple.
hdus_BANZAI_filtro_B = [hdu_BANZAI_189,hdu_BANZAI_193,hdu_BANZAI_197,hdu_BANZAI_201]
headers_BANZAI_filtro_B = [header_BANZAI_189,header_BANZAI_193,header_BANZAI_197,header_BANZAI_201]
datas_BANZAI_filtro_B = [data_BANZAI_189,data_BANZAI_193,data_BANZAI_197,data_BANZAI_201]

hdus_BANZAI_filtro_V = [hdu_BANZAI_190,hdu_BANZAI_194,hdu_BANZAI_198,hdu_BANZAI_202]
headers_BANZAI_filtro_V = [header_BANZAI_190,header_BANZAI_194,header_BANZAI_198,header_BANZAI_202]
datas_BANZAI_filtro_V = [data_BANZAI_190,data_BANZAI_194,data_BANZAI_198,data_BANZAI_202]

Set OBSGEO-B to   -31.272933 from OBSGEO-[XYZ].
Set OBSGEO-H to     1164.993 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -31.272933 from OBSGEO-[XYZ].
Set OBSGEO-H to     1164.993 from OBSGEO-[XYZ]'.


Para estimar la diferencia de flujo porcentual captada por el archivo BANZAI y el archivo reducido manualmente, se realizará la siguiente operación:
\begin{equation}
  \varepsilon_{err}=\left|\dfrac{flujo_{BANZAI}-flujo_{REDUCCION}}{flujo_{BANZAI}}\right|\cdot100\%,
\end{equation}
$\quad\quad\quad\quad$de donde se extraerá un valor de error para cada archivo de cada filtro, luego se calculará la media de cada valor de error en cada filtro.

In [19]:
valores_de_error_filtro_B = []
valores_de_error_filtro_V = []

for i in range(0,4):
  valores_de_error_filtro_B.append(np.abs(((np.array(datas_BANZAI_filtro_B[i])-np.array(campos_reducidos_filtro_B[i]))/(np.array(datas_BANZAI_filtro_B[i]))).mean())*100)

for i in range(0,4):
  valores_de_error_filtro_V.append(np.abs(((np.array(datas_BANZAI_filtro_V[i])-np.array(campos_reducidos_filtro_V[i]))/(np.array(datas_BANZAI_filtro_V[i]))).mean())*100)

error_porcentual_filtro_B = np.array(valores_de_error_filtro_B).mean()
error_porcentual_filtro_V = np.array(valores_de_error_filtro_V).mean()

print(f'El error porcentual en el filtro B corresponde a: {error_porcentual_filtro_B:.2f}, mientras que en el filtro V: {error_porcentual_filtro_V:.2f}')

El error porcentual en el filtro B corresponde a: 3.54, mientras que en el filtro V: 1.26


Dado esto, la diferencia porcentual entre los datos reducidos manualmente con los archivos BANZAI es de aproximadamente un $3.54\%$ para el filtro B, y un $1.26\%$ para el filtro V.

Se guardan los datos de campos reducidos en nuevos archivos FITS para su posterior uso.

In [21]:
etiquetas_archivos_campo_reducido_filtro_B = ['campo_reducido_189','campo_reducido_193','campo_reducido_197','campo_reducido_201']
etiquetas_archivos_campo_reducido_filtro_V = ['campo_reducido_190','campo_reducido_194','campo_reducido_198','campo_reducido_202']

# Filtro B
for i in range(0,4):
  fits.writeto(etiquetas_archivos_campo_reducido_filtro_B[i],campos_reducidos_filtro_B[i].data.astype('float32'), overwrite=True)

# Filtro V
for i in range(0,4):
  fits.writeto(etiquetas_archivos_campo_reducido_filtro_V[i],campos_reducidos_filtro_V[i].data.astype('float32'), overwrite=True)